In [2]:
import json

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [3]:
from sentence_transformers import SentenceTransformer

In [4]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [5]:
model.encode("test sentence!")

array([ 4.73348908e-02,  1.27067581e-01,  3.84838022e-02,  7.23323002e-02,
        3.31797544e-03,  6.13833033e-02,  6.92145452e-02, -3.69955897e-02,
       -2.15685386e-02,  5.20314649e-03,  6.65259808e-02, -7.02468827e-02,
        3.88374291e-02, -6.01203460e-03, -6.79618097e-04, -1.06657900e-01,
       -1.41483871e-03,  2.94834021e-02, -4.66492958e-02,  1.10237591e-03,
        7.50068924e-04, -1.48900365e-02,  5.91124445e-02, -4.67299949e-03,
       -2.62887590e-02,  5.87669052e-02,  2.24543922e-03,  4.08261530e-02,
        4.71807905e-02, -1.59825385e-02, -2.86675338e-02,  5.91224506e-02,
       -4.65365797e-02,  4.41080816e-02,  1.92513969e-02,  2.18564905e-02,
        5.51372953e-02,  2.41753124e-02,  7.44414842e-03, -1.32472850e-02,
       -4.55183387e-02, -1.38624296e-01,  2.90865880e-02,  2.29970384e-02,
        5.88652305e-02,  2.93165501e-02, -1.53530380e-02,  2.24739891e-02,
       -4.49639745e-02, -8.55540112e-02, -2.60863304e-02, -4.18343097e-02,
       -5.56011535e-02, -

Test Performance of 3 different vectors

In [6]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    question_text = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(question_text)

    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

100%|██████████| 948/948 [03:29<00:00,  4.53it/s]


In [8]:
query = 'I just discovered the course. Can I still join it?'
v_q = model.encode(query)

In [9]:
search_query = {
    "field": "question_vector",
    "query_vector": v_q,
    "k": 5,
    "num_candidates": 10000,
}

es_results = es_client.search(
    index=index_name,
    knn=search_query,
    source=["text", "section", "question", "course", "id"]
)

In [11]:
result_docs = []
    
for hit in es_results['hits']['hits']:
    result_docs.append(hit['_source'])

result_docs

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'id': '7842b56a'},
 {'question': 'Course

In [13]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [16]:
elastic_search_knn('question_vector', v_q, 'data-engineering-zoomcamp')

[{'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'id': '7842b56a'},
 {'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'id': 'a482086d'},
 {'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp',
  'section': '

In [17]:
import pandas as pd

In [18]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')
ground_truth = df_ground_truth.to_dict(orient='records')
ground_truth[0]

{'question': 'When does the course begin?',
 'course': 'data-engineering-zoomcamp',
 'document': 'c02e79ef'}

In [19]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)

In [20]:
def hit_rate(relevance_total):
    x = 0
    
    for line in relevance_total:
        if True in line:
            x += 1
    
    return x / len(relevance_total)

In [21]:
def mrr(relevance_total):
    score = 0
    
    for line in relevance_total:
        for i in range(len(line)):
            if line[i] == True:
                score = score + 1 / (i + 1)
    
    return score / len(relevance_total)

In [22]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)
    
    return {
        'hit_rate': hit_rate(relevance_total), 
        'mrr': mrr(relevance_total)
    }

In [23]:
evaluate(ground_truth, question_vector_knn)

100%|██████████| 4627/4627 [01:30<00:00, 51.04it/s]


{'hit_rate': 0.773071104387292, 'mrr': 0.6666810748505158}

In [30]:
def text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('text_vector', v_q, course)

In [31]:
text_vector_knn(ground_truth[0])

[{'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'id': 'c02e79ef'},
 {'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continu

In [32]:
evaluate(ground_truth, text_vector_knn)

100%|██████████| 4627/4627 [01:27<00:00, 52.65it/s]


{'hit_rate': 0.8286146531229739, 'mrr': 0.7062315395144454}

In [33]:
def question_text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)

In [34]:
evaluate(ground_truth, question_text_vector_knn)

100%|██████████| 4627/4627 [01:18<00:00, 58.68it/s]


{'hit_rate': 0.9172249837907932, 'mrr': 0.824306606152295}

In [35]:
def elastic_search_knn_combined(vector, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "course": course
                                }
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'question_vector') + 
                                    cosineSimilarity(params.query_vector, 'text_vector') + 
                                    cosineSimilarity(params.query_vector, 'question_text_vector') + 
                                    1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [36]:
def vector_combined_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn_combined(v_q, course)

evaluate(ground_truth, vector_combined_knn)

100%|██████████| 4627/4627 [01:22<00:00, 56.23it/s]


{'hit_rate': 0.9023125135076724, 'mrr': 0.804480945176861}